In [27]:
#import scrublet as scr
import numpy as np
import pandas as pd
import csv
import scanpy as sc 
import scanpy.external as sce
import phate
import pickle
import matplotlib
from matplotlib import pyplot as plt
import warnings
from scipy.stats import binom
from scipy.stats import multinomial
import seaborn as sns
from scipy.stats import hypergeom
import warnings
warnings.filterwarnings('ignore')
import itertools
import harmonypy as hm
from random import sample
from scipy.stats import linregress

from _util_updated import gen_sg_combos
from _util_updated import cluster_bias
from _util_updated import CB_Filter_NC

In [28]:
### Save Directory
PATH = ''

### Load Data 

In [29]:
### sgRNA DataFrame
SG_DF = pd.read_pickle('./DAP3_SGRNA_IND_072821.pkl')

### Single Cell AnnData
CM_Cells = sc.read_h5ad('./DAP3_SC_CM_Pseudo_PHATE_Man_080521.h5ad')

### CM SGRNA Processing (Subset sg cells, create sg dictionary)

In [30]:
### single cell dataset already selected for sg cells. just need to subset sgdf
SG_CM = SG_DF.loc[CM_Cells.obs.index]

In [31]:
## Unique target list
Target_List = []
for target in SG_CM.columns:
    Target_List.append(target.split(':')[0])
Unique_Target_List = np.unique(Target_List)

In [32]:
# Dictionary to get individual sg name
Target_Name = []
Target_Guides = []

for target in Unique_Target_List:
    Target_Name.append(target)
    if Target_List.count(target) == 1:
        Target_Guides.append(target)
    else:
        Target_Guide_Range = list(range(Target_List.count(target)))
        
        Subset_Guides = []
        for guide in Target_Guide_Range:
            Subset_Guides.append(target + ':' + str(guide + 1))
        Target_Guides.append(Subset_Guides)
Guide_Dictionary = dict(zip(Target_Name,Target_Guides))

In [33]:
### Make clean cell lists for each NC
sg_bool = SG_CM > 0
sg_pos = []
for cell in sg_bool.index:
    if sum(sg_bool.loc[cell]) == 1:
        sg_pos.append(cell)
SG_CM_Indi = SG_CM.loc[sg_pos]

NC_GAG_List = SG_CM_Indi[SG_CM_Indi['NC_GAG'] > 0].index.tolist()
NC_GFP_List = SG_CM_Indi[SG_CM_Indi['NC_GFP'] > 0].index.tolist()
NC_GFP_1_List = SG_CM_Indi[SG_CM_Indi['NC_GFP_1'] > 0].index.tolist()
NC_GFP_2_List = SG_CM_Indi[SG_CM_Indi['NC_GFP_2'] > 0].index.tolist()
NC_HS2_List = SG_CM_Indi[SG_CM_Indi['NC_HS2'] > 0].index.tolist()

NC_Guide_List= ['NC_GAG', 'NC_GFP', 'NC_GFP_1', 'NC_GFP_2', 'NC_HS2']
ALL_NC_LISTS = [NC_GAG_List,NC_GFP_List,NC_GFP_1_List,NC_GFP_2_List,NC_HS2_List]

In [34]:
New_NC_Name = ['NC:1', 'NC:2', 'NC:3', 'NC:4', 'NC:5']
NC_NAME_DICT = dict(zip(NC_Guide_List, New_NC_Name))
Col_Name = []
for col in SG_CM.columns:
    if col in NC_NAME_DICT.keys():
        Col_Name.append(NC_NAME_DICT[col])
    else:
        Col_Name.append(col)
SG_CM.columns = Col_Name

In [35]:
# Dictionary to get cells for any given guide
Target_K = []
Target_V = []

for target in SG_CM.columns:
    Target_K.append(target)
    Target_V.append(SG_CM[SG_CM[target] > 0 ].index.tolist())
SGRNA_CELLS_DIC = dict(zip(Target_K, Target_V))
Guide_Dictionary['NC'] = New_NC_Name

NC_DICT = dict(zip(New_NC_Name,ALL_NC_LISTS))

### Make NCs clean
for nc in New_NC_Name:
    SGRNA_CELLS_DIC[nc] = NC_DICT[nc]

In [39]:
for nc in NC_Guide_List:
    del Guide_Dictionary[nc]

In [41]:
### Save Guide Dictionary
a_file = open(PATH + "Guide_Dictionary.pkl", "wb")
pickle.dump(Guide_Dictionary, a_file)
a_file.close()

In [42]:
### Save Guide Dictionary
a_file = open(PATH + "CM_Large_sgRNA_Dic.pkl", "wb")
pickle.dump(SGRNA_CELLS_DIC, a_file)
a_file.close()

### Repeat for all cells

In [17]:
### Single Cell AnnData
SC_DF = sc.read_h5ad('./DAP3_SC_072821.h5ad')
SG_DF = pd.read_pickle('./DAP3_SGRNA_IND_072821.pkl')

In [18]:
### single cell dataset already selected for sg cells. just need to subset sgdf
SG_DF = SG_DF.loc[SC_DF.obs.index]

In [19]:
## Unique target list
Target_List = []
for target in SG_DF.columns:
    Target_List.append(target.split(':')[0])
Unique_Target_List = np.unique(Target_List)

In [20]:
# Dictionary to get individual sg name
Target_Name = []
Target_Guides = []

for target in Unique_Target_List:
    Target_Name.append(target)
    if Target_List.count(target) == 1:
        Target_Guides.append(target)
    else:
        Target_Guide_Range = list(range(Target_List.count(target)))
        
        Subset_Guides = []
        for guide in Target_Guide_Range:
            Subset_Guides.append(target + ':' + str(guide + 1))
        Target_Guides.append(Subset_Guides)
Guide_Dictionary = dict(zip(Target_Name,Target_Guides))

In [21]:
### Make clean cell lists for each NC
sg_bool = SG_DF > 0
sg_pos = []
for cell in sg_bool.index:
    if sum(sg_bool.loc[cell]) == 1:
        sg_pos.append(cell)
SG_DF_Indi = SG_DF.loc[sg_pos]

NC_GAG_List = SG_DF_Indi[SG_DF_Indi['NC_GAG'] > 0].index.tolist()
NC_GFP_List = SG_DF_Indi[SG_DF_Indi['NC_GFP'] > 0].index.tolist()
NC_GFP_1_List = SG_DF_Indi[SG_DF_Indi['NC_GFP_1'] > 0].index.tolist()
NC_GFP_2_List = SG_DF_Indi[SG_DF_Indi['NC_GFP_2'] > 0].index.tolist()
NC_HS2_List = SG_DF_Indi[SG_DF_Indi['NC_HS2'] > 0].index.tolist()

NC_Guide_List= ['NC_GAG', 'NC_GFP', 'NC_GFP_1', 'NC_GFP_2', 'NC_HS2']
ALL_NC_LISTS = [NC_GAG_List,NC_GFP_List,NC_GFP_1_List,NC_GFP_2_List,NC_HS2_List]

In [22]:
New_NC_Name = ['NC:1', 'NC:2', 'NC:3', 'NC:4', 'NC:5']
NC_NAME_DICT = dict(zip(NC_Guide_List, New_NC_Name))
Col_Name = []
for col in SG_DF.columns:
    if col in NC_NAME_DICT.keys():
        Col_Name.append(NC_NAME_DICT[col])
    else:
        Col_Name.append(col)
SG_DF.columns = Col_Name

In [23]:
# Dictionary to get cells for any given guide
Target_K = []
Target_V = []

for target in SG_DF.columns:
    Target_K.append(target)
    Target_V.append(SG_DF[SG_DF[target] > 0 ].index.tolist())
SGRNA_CELLS_DIC = dict(zip(Target_K, Target_V))
Guide_Dictionary['NC'] = New_NC_Name

NC_DICT = dict(zip(New_NC_Name,ALL_NC_LISTS))

### Make NCs clean
for nc in New_NC_Name:
    SGRNA_CELLS_DIC[nc] = NC_DICT[nc]

In [24]:
### Save Guide Dictionary
a_file = open(PATH + "ALL_Large_sgRNA_Dic.pkl", "wb")
pickle.dump(SGRNA_CELLS_DIC, a_file)
a_file.close()